<a href="https://colab.research.google.com/github/muntakim1/data-science/blob/main/Water_Segment_Yolov8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir /root/.kaggle

In [ ]:
!cp ./kaggle.json /root/.kaggle

In [ ]:
!kaggle datasets download -d gvclsu/water-segmentation-dataset

100% 4.77G/4.77G [02:44<00:00, 41.0MB/s]
100% 4.77G/4.77G [02:44<00:00, 31.1MB/s]


In [ ]:
!unzip water-segmentation-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: water_v2/water_v2/Annotations/ADE20K/ADE_val_00001289.png  
  inflating: water_v2/water_v2/Annotations/ADE20K/ADE_val_00001354.png  
  inflating: water_v2/water_v2/Annotations/ADE20K/ADE_val_00001355.png  
  inflating: water_v2/water_v2/Annotations/ADE20K/ADE_val_00001357.png  
  inflating: water_v2/water_v2/Annotations/ADE20K/ADE_val_00001361.png  
  inflating: water_v2/water_v2/Annotations/ADE20K/ADE_val_00001377.png  
  inflating: water_v2/water_v2/Annotations/ADE20K/ADE_val_00001399.png  
  inflating: water_v2/water_v2/Annotations/ADE20K/ADE_val_00001427.png  
  inflating: water_v2/water_v2/Annotations/ADE20K/ADE_val_00001439.png  
  inflating: water_v2/water_v2/Annotations/ADE20K/ADE_val_00001441.png  
  inflating: water_v2/water_v2/Annotations/ADE20K/ADE_val_00001445.png  
  inflating: water_v2/water_v2/Annotations/ADE20K/ADE_val_00001447.png  
  inflating: water_v2/water_v2/Annotations/ADE20K/ADE_val_00001448.png  


In [ ]:
!pip install ultralytics==8.0.58

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.8/486.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 12.1 MB/s eta 0:00:00


In [ ]:
!rm -r /content/water_v1/water_v1/Labels

rm: cannot remove '/content/water_v1/water_v1/Labels': No such file or directory


In [ ]:
import os

import cv2


input_dir = '/content/water_v1/water_v1/Annotations/'
output_dir = '/content/water_v1/water_v1/Labels/'

for j in os.listdir(input_dir):

  for i in os.listdir(os.path.join(input_dir,j)):

      image_path = os.path.join(os.path.join(input_dir,j), i)
      # load the binary mask and get its contours
      mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
      _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

      H, W = mask.shape
      contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

      # convert the contours to polygons
      polygons = []
      for cnt in contours:
          if cv2.contourArea(cnt) > 200:
              polygon = []
              for point in cnt:
                  x, y = point[0]
                  polygon.append(x / W)
                  polygon.append(y / H)
              polygons.append(polygon)
      #print the polygons
      if not os.path.exists(os.path.join(output_dir,j)):
        os.makedirs(os.path.join(output_dir,j))
      with open('{}.txt'.format(os.path.join(output_dir, os.path.join(j,i[:-4]))), 'w') as f:
          for polygon in polygons:
              for p_, p in enumerate(polygon):
                  if p_ == len(polygon) - 1:
                      f.write('{}\n'.format(p))
                  elif p_ == 0:
                      f.write('0 {} '.format(p))
                  else:
                      f.write('{} '.format(p))

          f.close()

In [ ]:
!rm -r /content/dataset/

rm: cannot remove '/content/dataset/': No such file or directory


In [ ]:
!mkdir dataset

!mkdir dataset/images
!mkdir dataset/images/train
!mkdir dataset/images/val

!mkdir dataset/labels
!mkdir dataset/labels/train
!mkdir dataset/labels/val



In [ ]:
import os
import shutil

input_dir = '/content/water_v1/water_v1/Labels/'

for j in os.listdir(input_dir):
  if j=="stream2" or j=="stream1" or j=="stream0":
    for i in os.listdir(os.path.join(input_dir,j)):
      shutil.copy(os.path.join(os.path.join(input_dir,j),i), "/content/dataset/labels/val/"+i)
  else:
    for i in os.listdir(os.path.join(input_dir,j)):
      shutil.copy(os.path.join(os.path.join(input_dir,j),i), "/content/dataset/labels/train/"+i)



In [ ]:
import os
import shutil

input_dir = '/content/water_v1/water_v1/JPEGImages'

for j in os.listdir(input_dir):
  if j=="stream2" or j=="stream1" or j=="stream0":
    for i in os.listdir(os.path.join(input_dir,j)):
      shutil.copy(os.path.join(os.path.join(input_dir,j),i), "/content/dataset/images/val/"+i)
  else:
    for i in os.listdir(os.path.join(input_dir,j)):
      shutil.copy(os.path.join(os.path.join(input_dir,j),i), "/content/dataset/images/train/"+i)



In [ ]:
!rm -r runs

rm: cannot remove 'runs': No such file or directory


In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8m-seg.pt')

results = model.train(data='/content/custom-segmet.yaml', epochs=5,imgsz=640)

New https://pypi.org/project/ultralytics/8.0.178 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.58 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=segment, mode=train, model=yolov8m-seg.pt, data=/content/custom-segmet.yaml, epochs=5, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None

In [ ]:
from ultralytics import YOLO

import cv2


model_path = '/content/runs/segment/train/weights/best.pt'

image_path = '/content/water_v1/water_v1/JPEGImages/canal0/1.png'


img = cv2.imread(image_path)
H, W, _ = img.shape

model = YOLO(model_path)

results = model(img,save=True)

# for result in results:
#     for j, mask in enumerate(result.masks.data):

#         mask = mask.cpu().numpy() * 255

#         mask = cv2.resize(mask, (W, H))

#         cv2.imwrite('./output.png', mask)


0: 384x640 2 waters, 36.8ms
Speed: 0.5ms preprocess, 36.8ms inference, 5.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/segment/predict
